In [1]:
import mujoco
import numpy as np
import pinocchio as pin
from robot_descriptions.skydio_x2_description import URDF_PATH
from robot_descriptions.skydio_x2_mj_description import MJCF_PATH

from mujoco_sysid import parameters
from mujoco_sysid.utils import muj2pin

model = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(model)

for act_id in range(4):
    model.actuator(act_id).ctrlrange = np.array([-1e4, 1e4])

model.opt.timestep = 1e-3

In [2]:
sparceM = np.zeros((6, 6))

mujoco.mj_step(model, mjdata)
mujoco.mj_fullM(model, sparceM, mjdata.qM)

mSrc = sparceM[3:6, :3]
mrc = np.array([mSrc[2, 1], mSrc[0, 2], mSrc[1, 0]])
Ib = sparceM[3:6, 3:6]
vechIb = np.array([Ib[0, 0], Ib[0, 1], Ib[1, 1], Ib[0, 2], Ib[1, 2], Ib[2, 2]])
skydio_parameters = np.array([sparceM[0, 0], *mrc, *vechIb])

with np.printoptions(precision=5, suppress=True):
    print(skydio_parameters)

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [3]:
with np.printoptions(precision=5, suppress=True):
    print(parameters.get_dynamic_parameters(model, 1))

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [4]:
pinmodel = pin.buildModelFromUrdf(URDF_PATH)

with np.printoptions(precision=5, suppress=True):
    print(pinmodel.inertias[1].toDynamicParameters())

[ 1.325    0.       0.       0.0715   0.04051  0.       0.02927 -0.0021
  0.       0.06053]


In [5]:
def mj_bodyRegressor(mj_model: mujoco.MjModel, mj_data: mujoco.MjData, body_id: int) -> np.ndarray:
    """
    Computes the body regressor matrix for a specific body in a MuJoCo model.

    Args:
        mj_model (mujoco.MjModel): MuJoCo model.
        mj_data (mujoco.MjData): MuJoCo data.
        body_id (int): ID of the body for which to compute the regressor matrix.

    Returns:
        np.ndarray: Body regressor matrix of shape (6, 10).
    """
    velocity = np.zeros(6)
    accel = np.zeros(6)
    _cross = np.zeros(3)

    mujoco.mj_objectVelocity(mj_model, mj_data, 2, body_id, velocity, 1)
    mujoco.mj_rnePostConstraint(mj_model, mj_data)
    mujoco.mj_objectAcceleration(mj_model, mj_data, 2, body_id, accel, 1)

    v, w = velocity[3:], velocity[:3]
    dv, dw = accel[3:], accel[:3]  # dv - classical acceleration, already contains g
    mujoco.mju_cross(_cross, w, v)
    # dv -= _cross
    print(f"regressor dv: {dv} dw: {dw}")

    v1, v2, v3 = v
    v4, v5, v6 = w

    a1, a2, a3 = dv
    a4, a5, a6 = dw

    # fmt: off
    Y = np.array(
        [
            [a1 - v2 * v6 + v3 * v5, -(v5**2) - v6**2, -a6 + v4 * v5, a5 + v4 * v6, 0, 0, 0, 0, 0, 0],
            [a2 + v1 * v6 - v3 * v4, a6 + v4 * v5, -(v4**2) - v6**2, -a4 + v5 * v6, 0, 0, 0, 0, 0, 0],
            [a3 - v1 * v5 + v2 * v4, -a5 + v4 * v6, a4 + v5 * v6, -(v4**2) - v5**2, 0, 0, 0, 0, 0, 0],
            [0, 0, a3 - v1 * v5 + v2 * v4, -a2 - v1 * v6 + v3 * v4, a4, a5 - v4 * v6, -v5 * v6, a6 + v4 * v5, v5**2 - v6**2, v5 * v6],
            [0, -a3 + v1 * v5 - v2 * v4, 0, a1 - v2 * v6 + v3 * v5, v4 * v6, a4 + v5 * v6, a5, -(v4**2) + v6**2, a6 - v4 * v5, -v4 * v6],
            [0, a2 + v1 * v6 - v3 * v4, -a1 + v2 * v6 - v3 * v5, 0, -v4 * v5, v4**2 - v5**2, v4 * v5, a4 - v5 * v6, a5 + v4 * v6, a6],
        ]
    )
    # fmt: on

    return Y

In [6]:
import json

with open("../data/ltv_lqr_traj.json") as f:
    data = json.load(f)

In [7]:
np.random.seed(0)

# traj_idx = 2

# q = data["q"][traj_idx]
# v = data["v"][traj_idx]
# a = data["dv"][traj_idx]
# ctrl = data["u"][traj_idx]

q = np.random.randn(model.nq)
q[3:7] = q[3:7] / np.linalg.norm(q[3:7])
v = np.random.randn(model.nv)
a = np.random.randn(model.nv)
ctrl = np.random.randn(model.nu)

mjdata = mujoco.MjData(model)
mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = a
# mjdata.ctrl[:] = ctrl

# mujoco.mj_step(model, mjdata)
# mujoco.mj_forward(model, mjdata)
mujoco.mj_inverse(model, mjdata)

In [8]:
regressor = mj_bodyRegressor(model, mjdata, 1)

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(regressor)

regressor dv: [7.86184879 6.34771817 1.00187616] dw: [ 1.49407907 -0.20515826  0.3130677 ]
[[ 7.84074 -2.69409 -0.10359 -0.09554  0.       0.       0.       0.       0.       0.     ]
 [ 6.48332  0.52255 -0.59993 -0.38732  0.       0.       0.       0.       0.       0.     ]
 [ 0.74675  0.31478  2.60084 -2.13566  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.74675 -6.48332  1.49408 -0.31478 -1.10676  0.52255  1.53573  1.10676]
 [ 0.      -0.74675  0.       7.84074  0.10962  2.60084 -0.20516  0.55843  0.10359 -0.10962]
 [ 0.       6.48332 -7.84074  0.      -0.20948 -2.09416  0.20948  0.38732 -0.09554  0.31307]]


In [9]:
from mujoco_sysid.regressors import joint_body_regressor

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(joint_body_regressor(model, mjdata, 1))

[[ 7.86185 -2.69409 -0.10359 -0.09554  0.       0.       0.       0.       0.       0.     ]
 [ 6.34772  0.52255 -0.59993 -0.38732  0.       0.       0.       0.       0.       0.     ]
 [ 1.00188  0.31478  2.60084 -2.13566  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       1.00188 -6.34772  1.49408 -0.31478 -1.10676  0.52255  1.53573  1.10676]
 [ 0.      -1.00188  0.       7.86185  0.10962  2.60084 -0.20516  0.55843  0.10359 -0.10962]
 [ 0.       6.34772 -7.86185  0.      -0.20948 -2.09416  0.20948  0.38732 -0.09554  0.31307]]


In [10]:
force = mjdata.qfrc_actuator.copy()
force[:3] = mjdata.xmat[1].reshape(3, 3).T @ force[:3]

with np.printoptions(precision=5, suppress=True):
    print(force)

[0. 0. 0. 0. 0. 0.]


In [11]:
with np.printoptions(precision=5, suppress=True):
    print(regressor @ skydio_parameters)

[10.38215  8.56271  0.83674 -0.36953  0.55124  0.01578]


In [12]:
np.linalg.norm(force)

0.0

### Set same values for pinocchio and compare

In [13]:
pinq, pinv = muj2pin(mjdata.qpos, mjdata.qvel)

_, pindv = muj2pin(mjdata.qpos, mjdata.qacc)

pinq, pinv, pindv

(array([ 1.76405235,  0.40015721,  0.97873798,  0.58002289, -0.30352125,
         0.2950768 ,  0.69597269]),
 array([0.21085653, 0.35761936, 0.17263309, 0.14404357, 1.45427351,
        0.76103773]),
 array([ 0.35928247,  0.18472281, -0.3999394 ,  1.49407907, -0.20515826,
         0.3130677 ]))

In [14]:
pindata = pin.Data(pinmodel)
tau = pin.rnea(pinmodel, pindata, pinq, pinv, pindv)

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(pin.jointBodyRegressor(pinmodel, pindata, 1))

[[ 7.84074 -2.69409 -0.10359 -0.09554  0.       0.       0.       0.       0.       0.     ]
 [ 6.48332  0.52255 -0.59993 -0.38732  0.       0.       0.       0.       0.       0.     ]
 [ 0.74675  0.31478  2.60084 -2.13566  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.74675 -6.48332  1.49408 -0.31478 -1.10676  0.52255  1.53573  1.10676]
 [ 0.      -0.74675  0.       7.84074  0.10962  2.60084 -0.20516  0.55843  0.10359 -0.10962]
 [ 0.       6.48332 -7.84074  0.      -0.20948 -2.09416  0.20948  0.38732 -0.09554  0.31307]]


In [15]:
pindata.a_gf[1], pindata.v[1]

(  v = 7.86185 6.34772 1.00188
   w =   1.49408 -0.205158  0.313068,
   v = 0.210857 0.357619 0.172633
   w = 0.144044  1.45427 0.761038)

In [16]:
regressor = mj_bodyRegressor(model, mjdata, 1)

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(regressor)

regressor dv: [7.86184879 6.34771817 1.00187616] dw: [ 1.49407907 -0.20515826  0.3130677 ]
[[ 7.84074 -2.69409 -0.10359 -0.09554  0.       0.       0.       0.       0.       0.     ]
 [ 6.48332  0.52255 -0.59993 -0.38732  0.       0.       0.       0.       0.       0.     ]
 [ 0.74675  0.31478  2.60084 -2.13566  0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.74675 -6.48332  1.49408 -0.31478 -1.10676  0.52255  1.53573  1.10676]
 [ 0.      -0.74675  0.       7.84074  0.10962  2.60084 -0.20516  0.55843  0.10359 -0.10962]
 [ 0.       6.48332 -7.84074  0.      -0.20948 -2.09416  0.20948  0.38732 -0.09554  0.31307]]


In [17]:
with np.printoptions(precision=5, suppress=True, linewidth=400):
    print("regressor", pin.jointBodyRegressor(pinmodel, pindata, 1) @ pinmodel.inertias[1].toDynamicParameters())
    print("rnea", tau)

regressor [10.38215  8.56271  0.83674 -0.36953  0.55124  0.01578]
rnea [10.38215  8.56271  0.83674 -0.36953  0.55124  0.01578]


In [18]:
np.linalg.norm(tau)

13.499994938181835